In [ ]:
import kagglehub
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from category_encoders.count import CountEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
! pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
kagglehub.dataset_download("undefinenull/million-song-dataset-spotify-lastfm")

100%|██████████| 639M/639M [00:15<00:00, 42.3MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1'

In [ ]:
data_path = Path("/root/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1")


songs_data_path = data_path / 'Music Info.csv'
users_data_path = data_path / 'User Listening History.csv'

In [ ]:
# load the songs data
df_songs = pd.read_csv(songs_data_path)
df_songs.head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


#EDA


In [ ]:
df_songs.shape

(50683, 21)

In [ ]:
df_songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   track_id             50683 non-null  object 
 1   name                 50683 non-null  object 
 2   artist               50683 non-null  object 
 3   spotify_preview_url  50683 non-null  object 
 4   spotify_id           50683 non-null  object 
 5   tags                 49556 non-null  object 
 6   genre                22348 non-null  object 
 7   year                 50683 non-null  int64  
 8   duration_ms          50683 non-null  int64  
 9   danceability         50683 non-null  float64
 10  energy               50683 non-null  float64
 11  key                  50683 non-null  int64  
 12  loudness             50683 non-null  float64
 13  mode                 50683 non-null  int64  
 14  speechiness          50683 non-null  float64
 15  acousticness         50683 non-null 

In [ ]:
#calculate missing values and missing percentage
#can use KNN imputer for imputation technique
missing_values=df_songs.isnull().mean()*100

,0
track_id,0.000000
name,0.000000
artist,0.000000
spotify_preview_url,0.000000
spotify_id,0.000000
tags,2.223625
genre,55.906320
year,0.000000
duration_ms,0.000000
danceability,0.000000


In [ ]:
#Drop duplicate rows
df_songs.drop_duplicates(subset=['spotify_id','duration_ms','year'], inplace=True)
df_songs['spotify_id'].duplicated().sum()

np.int64(0)

In [ ]:
#Reset index
df_songs.reset_index(drop=True, inplace=True)

In [ ]:
#drop the unnecessary column
new_df=df_songs.drop(columns=['genre','track_id','name','spotify_preview_url','spotify_id','index_column'], axis=1, inplace=False)
new_df.shape

(50674, 16)

In [ ]:
#replace missing values
new_df['tags'].fillna('no tags', inplace=True)

In [ ]:
#Convert into lower case
new_df['artist']=new_df['artist'].str.lower()
new_df['artist']


,artist
0,the killers
1,oasis
2,nirvana
3,franz ferdinand
4,radiohead
...,...
50669,アンティック-珈琲店-
50670,acidman
50671,coldrain
50672,アンティック-珈琲店-


In [ ]:
new_df['artist'].nunique()
new_df['year'].nunique()

75

In [ ]:
new_df.describe()

,year,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,50674.000000,5.067400e+04,50674.000000,50674.000000,50674.000000,50674.000000,50674.000000,50674.000000,50674.000000,50674.000000,50674.000000,50674.000000,50674.000000,50674.000000
mean,2004.017109,2.511536e+05,0.493522,0.686507,5.312507,-8.291007,0.631073,0.076026,0.213798,0.225299,0.215439,0.433113,123.508794,3.898153
std,8.860497,1.075892e+05,0.178833,0.251803,3.567967,4.548359,0.482519,0.076012,0.302839,0.337067,0.184708,0.258767,29.622349,0.419686
min,1900.000000,1.439000e+03,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2001.000000,1.927330e+05,0.364000,0.514000,2.000000,-10.375000,0.000000,0.035200,0.001400,0.000018,0.098400,0.214000,100.682500,4.000000
50%,2006.000000,2.349330e+05,0.497000,0.744000,5.000000,-7.199500,1.000000,0.048200,0.039900,0.005630,0.138000,0.405000,121.989000,4.000000
75%,2009.000000,2.881828e+05,0.621000,0.905000,9.000000,-5.089000,1.000000,0.083500,0.340000,0.441000,0.289000,0.634000,141.642250,4.000000
max,2022.000000,3.816373e+06,0.986000,1.000000,11.000000,3.642000,1.000000,0.954000,0.996000,0.999000,0.999000,0.993000,238.895000,5.000000


In [ ]:
# value counts for the tags

(
    new_df
    .loc[:,'tags']
    .str.lower()
    .str.split(',')
    .explode()
    .str.strip()
    .value_counts()
)

,count
tags,
rock,10681
indie,7284
electronic,6592
alternative,6271
pop,4650
...,...
dark_ambient,602
japanese,489
polish,411


In [ ]:
# value counts for the tags

(
    new_df
    .loc[:,'tags']
    .str.lower()
    .str.split(',')
    .explode()
    .str.strip()
    .value_counts()
    [lambda x: x >= 1000]
)

,count
tags,
rock,10681
indie,7284
electronic,6592
alternative,6271
pop,4650
...,...
ska,1088
gothic_metal,1072
grindcore,1040


In [ ]:
# cols to transform
frequency_encode_cols = ['year']
ohe_cols = ['artist',"time_signature","key"]
tfidf_col = 'tags'
standard_scale_cols = ["duration_ms","loudness","tempo"]
min_max_scale_cols = ["danceability","energy","speechiness","acousticness","instrumentalness","liveness","valence"]

In [ ]:
# transform the data
transformer = ColumnTransformer(transformers=[
    ("frequency_encode", CountEncoder(normalize=True,return_df=True), frequency_encode_cols),
    ("ohe", OneHotEncoder(handle_unknown="ignore"), ohe_cols),
    ("tfidf", TfidfVectorizer(max_features=85), tfidf_col),
    ("standard_scale", StandardScaler(), standard_scale_cols),
    ("min_max_scale", MinMaxScaler(), min_max_scale_cols)
],remainder='passthrough',n_jobs=-1,force_int_remainder_cols=False)

transformer

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('frequency_encode',
                                 CountEncoder(combine_min_nan_groups=True,
                                              normalize=True),
                                 ['year']),
                                ('ohe', OneHotEncoder(handle_unknown='ignore'),
                                 ['artist', 'time_signature', 'key']),
                                ('tfidf', TfidfVectorizer(max_features=85),
                                 'tags'),
                                ('standard_scale', StandardScaler(),
                                 ['duration_ms', 'loudness', 'tempo']),
                                ('min_max_scale', MinMaxScaler(),
                                 ['danceability', 'energy', 'speechiness',
                                  'acousticness', 'instrumentalness',
                                  'liveness', 'valence'])])

In [ ]:
# fit the transformer
transformer.fit(new_df)

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('frequency_encode',
                                 CountEncoder(combine_min_nan_groups=True,
                                              normalize=True),
                                 ['year']),
                                ('ohe', OneHotEncoder(handle_unknown='ignore'),
                                 ['artist', 'time_signature', 'key']),
                                ('tfidf', TfidfVectorizer(max_features=85),
                                 'tags'),
                                ('standard_scale', StandardScaler(),
                                 ['duration_ms', 'loudness', 'tempo']),
                                ('min_max_scale', MinMaxScaler(),
                                 ['danceability', 'energy', 'speechiness',
                                  'acousticness', 'instrumentalness',
                                  'liveness', 'valence'])])

In [ ]:
transformed_df = transformer.transform(new_df)

In [ ]:
transformed_df.shape
type(transformed_df)

scipy.sparse._csr.csr_matrix

In [ ]:
df_songs[df_songs['artist']=='Shakira'].head()

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,index_column
1025,TRLWDVU128F932B093,"Whenever, Wherever",Shakira,https://p.scdn.co/mp3-preview/09ddeb4ae33ee6e8...,07PHBDuUmOeZ7jeKSbAbKi,"rock, pop, female_vocalists, singer_songwriter...",NaN,2012,196826,0.787,...,-4.967,0,0.0474,0.298,0.000005,0.2060,0.860,107.674,4,None
2068,TRILOWN128F426080F,Underneath Your Clothes,Shakira,https://p.scdn.co/mp3-preview/6c5a56058ce04371...,07qRl4PT2lA6O3KN40McLz,"rock, pop, female_vocalists, singer_songwriter...",Pop,2013,224893,0.707,...,-5.293,1,0.0298,0.691,0.000000,0.1030,0.407,82.784,4,None
2205,TRXLMFJ12903CC06F7,She Wolf,Shakira,https://p.scdn.co/mp3-preview/4dc802fd3b06fcb5...,075xFXR0JDBwFPVinG1ig5,"electronic, pop, female_vocalists, experimenta...",NaN,2009,187866,0.857,...,-6.480,1,0.0428,0.323,0.003220,0.3140,0.868,121.994,4,None
2469,TRSQAWU128F92EA20F,La Tortura,Shakira,https://p.scdn.co/mp3-preview/e62ae4649f029729...,0ofDrTTcinCUxm7wqCLPQa,"electronic, pop, female_vocalists, singer_song...",Latin,2011,213106,0.741,...,-5.904,1,0.0421,0.023,0.000788,0.1200,0.834,100.001,4,None
3373,TRINUNP12903CD84D9,Did It Again,Shakira,https://p.scdn.co/mp3-preview/5477eae2283113ff...,0eMNEdcC5OImvrfn79J9dU,"electronic, pop, female_vocalists, experimenta...",NaN,2009,227333,0.869,...,-5.069,0,0.0896,0.509,0.000000,0.0741,0.599,137.955,4,None


In [ ]:
# build input vector
df_songs[df_songs["name"] == "Whenever, Wherever"]
song_input = new_df[df_songs["name"] == "Whenever, Wherever"]
song_input

,artist,tags,year,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1025,shakira,"rock, pop, female_vocalists, singer_songwriter...",2012,196826,0.787,0.828,1,-4.967,0,0.0474,0.298,0.000005,0.206,0.86,107.674,4


In [ ]:
# input vector to calculate similarity
input_vector = transformer.transform(song_input)
input_vector

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 20 stored elements and shape (1, 8431)>

In [ ]:
# calculate the similarity matrix
similarity_scores = cosine_similarity(transformed_df,input_vector)
similarity_scores

array([[0.99999914],
       [0.99999847],
       [0.99999921],
       ...,
       [0.99999877],
       [0.9999992 ],
       [0.99999891]])

In [ ]:
similarity_scores.shape

(50674, 1)

In [ ]:
top_10_songs_indexes = np.argsort(similarity_scores.ravel())[-11:][::-1]
top_10_songs_indexes

array([ 1025, 12305,  6046,  6129, 17241,  6133,  7172,  6121,  6526,
       38383,  6287])

In [ ]:
top_10_songs_names = df_songs.iloc[top_10_songs_indexes]
top_10_songs_names

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,index_column
1025,TRLWDVU128F932B093,"Whenever, Wherever",Shakira,https://p.scdn.co/mp3-preview/09ddeb4ae33ee6e8...,07PHBDuUmOeZ7jeKSbAbKi,"rock, pop, female_vocalists, singer_songwriter...",NaN,2012,196826,0.787,...,-4.967,0,0.0474,0.29800,0.000005,0.2060,0.860,107.674,4,None
12305,TRYFVKK128F4240FE8,Why Wait,Shakira,https://p.scdn.co/mp3-preview/d78c90c5cb5626be...,0HiJFRxWme9myvUiDlqQ8q,"pop, experimental, singer_songwriter, dance",NaN,2001,221240,0.887,...,-5.535,0,0.0431,0.14400,0.000590,0.1230,0.399,129.943,4,None
6046,TRAAKDG128F42A0ECB,Hips Don't Lie,Shakira,https://p.scdn.co/mp3-preview/3859547944f57cfb...,01Yj2MCGpjZs34PRlGgz4K,"pop, female_vocalists, singer_songwriter, danc...",Pop,2001,217453,0.777,...,-5.867,0,0.0734,0.28400,0.000000,0.4300,0.760,100.003,4,None
6129,TRBAUVN128F932FEF8,Oops!...I Did It Again,Britney Spears,https://p.scdn.co/mp3-preview/7fb86827422540ad...,095uakqDYR50Uza0mxvPWB,"pop, female_vocalists, dance, 00s",Pop,2014,211786,0.751,...,-5.351,0,0.0435,0.34000,0.000018,0.2550,0.886,95.045,4,None
17241,TRMBDIR128F4279C1F,Perfect Lover,Britney Spears,https://p.scdn.co/mp3-preview/52671e54d36f077e...,1BhxPx4evrx8X02RHGrLdi,"pop, dance, rnb, 00s",Rock,2007,182680,0.718,...,-3.959,0,0.0360,0.35300,0.000390,0.1020,0.805,117.067,4,None
6133,TRDXCSH128F92ED4A1,Bootylicious,Destiny's Child,https://p.scdn.co/mp3-preview/7e327ccb1e4c52b2...,09mkdGhqb5ySKVsnkx9hy2,"pop, female_vocalists, dance, soul, hip_hop, r...",NaN,2001,207733,0.835,...,-4.364,0,0.2840,0.00247,0.000000,0.1710,0.586,103.358,4,None
7172,TRLQBEN128F92E7415,Wild Things,Alessia Cara,https://p.scdn.co/mp3-preview/c13f00088525d0b2...,0RgHkNpqtAHGGBVro6mmqD,"pop, female_vocalists",Country,2016,188493,0.741,...,-5.362,0,0.1080,0.01950,0.000000,0.0822,0.735,107.960,4,None
6121,TRGZIMZ128F930A016,La Isla Bonita,Madonna,https://p.scdn.co/mp3-preview/d8f3cafe99c1f0cd...,0rpndqrkU9y9nckNCfjcq6,"pop, female_vocalists, dance, 80s",NaN,2009,242946,0.708,...,-4.736,0,0.0362,0.39200,0.000001,0.0561,0.968,99.953,4,None
6526,TRXHWIA128E078A9BB,Cruel Summer,Bananarama,https://p.scdn.co/mp3-preview/47d13ef240a58bef...,0BmUCeyXpTrUVahHKVFxuB,"pop, female_vocalists, dance, 80s, new_wave",NaN,1984,209573,0.665,...,-5.828,0,0.0278,0.25000,0.008550,0.0537,0.932,108.429,4,None
38383,TRWUWRZ128F42ADA4A,Dreams for Plans,Shakira,https://p.scdn.co/mp3-preview/6e2c021846087a88...,2ObxMmMaDINr0ynkqW2BlY,"pop, female_vocalists, guitar, pop_rock",Pop,2005,242760,0.689,...,-7.427,1,0.0286,0.18000,0.000038,0.0844,0.548,96.098,4,None


In [ ]:
def recommend(song_name, songs_data, transformed_data, k=10):
    song_input = songs_data[songs_data["name"] == song_name]
    input_vector = transformer.transform(song_input)
    similarity_scores = cosine_similarity(transformed_data, input_vector)